# Softmax Classification

In [73]:
import numpy as np
import tensorflow as tf
tf.random.set_seed(123)

#### 데이터 셋 
* 리스트 형태로 작성 

In [74]:
# 1 데이터 
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]

# one-hot encoding
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#### 데이터 타입 변환 
* 리스트 --> numpy의 ndarray로 변환 

In [75]:
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)
print(x_data.shape)
print(y_data.shape)

nb_classes = 3  # 클래스의 종류 3개

(8, 4)
(8, 3)


In [76]:
W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]
print(variables)

[<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[-0.8980837 , -1.8259144 , -0.44441807],
       [-1.4882947 , -0.7855463 ,  0.19619656],
       [ 0.17604657, -1.5252507 ,  0.635294  ],
       [ 0.66812044,  1.4230527 ,  0.04561644]], dtype=float32)>, <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([ 0.33875433,  0.3449861 , -0.6605785 ], dtype=float32)>]


#### 가설 

In [77]:
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)
print(hypothesis(x_data))

tf.Tensor(
[[6.2787034e-02 3.9538752e-02 8.9767426e-01]
 [1.5100013e-01 1.3186018e-03 8.4768128e-01]
 [2.8089932e-01 4.3899636e-03 7.1471071e-01]
 [1.5661684e-01 6.8538109e-05 8.4331471e-01]
 [2.9546865e-05 1.4179445e-05 9.9995625e-01]
 [1.9907025e-01 6.0536587e-03 7.9487610e-01]
 [1.8745963e-04 1.7292357e-05 9.9979526e-01]
 [4.0955692e-05 2.9611417e-06 9.9995613e-01]], shape=(8, 3), dtype=float32)


#### Softmax one-hot 테스트
* 데이터를 생성할때 numpy의 ndarray로 생성하고 타입을 설정해야 한다.
* hypothesis(data)을 사용하면 가설값을 가져온다. [[0.0479028  0.9303597  0.02173743]] - 3가지 값을 가져온다. 
* tf.argmax(hypothesis(data), 1)은 열에서 가장 큰 원소의 위치를 반환 한다. [1] 윗 값 중에 가장 큰 인덱스 1이 반환 된다. 

In [78]:
data = np.array([[3., 2., 1., 8.]], dtype=np.float32)
print(hypothesis(data))  # 가설값 출력 
print(tf.argmax(hypothesis(data), 1)) # 가설값 중에 가장 큰 원소의 위치 출력 

tf.Tensor([[0.0479028  0.9303597  0.02173743]], shape=(1, 3), dtype=float32)
tf.Tensor([1], shape=(1,), dtype=int64)


#### cost 함수 (비용함수)
* X : 입력 데이터 
* Y : 정답 라벨 
* hypothesis(X) : 입력 데이터에 대한 가설값 
* -tf.reduce_sum(Y * tf.math.log(logits), axis=1) : 각 행에 대해 비용 합계
* tf.reduce_mean : 비용합계에 대한 평균(평균 비용) - 스칼라 값으로 반환 

In [79]:
def cost_fn(X, Y):
    logits = hypothesis(X)  # x에 대한 가설값 
    cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(logits), axis=1))  # 각 행에 대해 비용을 구한 후 함계 반환 -> 평균 
    return cost

print(cost_fn(x_data, y_data))

tf.Tensor(5.6441216, shape=(), dtype=float32)


#### 미분 연습하기 

In [80]:
x = tf.constant(3.0)
with tf.GradientTape() as tape:
    tape.watch(x)
    y = x * x 
dy_dx = tape.gradient(y, x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


####  grad(기울기 계산) 함수
* variables = [W, b]

In [81]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)  # 비용 계산 
        grads = tape.gradient(loss, variables) 
    return grads

print(grad_fn(x_data, y_data))

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.0041601 , -0.74228656,  0.7381264 ],
       [-1.4857689 , -1.2378517 ,  2.7236207 ],
       [-1.23269   , -1.8640661 ,  3.0967562 ],
       [-1.1915689 , -1.9879254 ,  3.1794944 ]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.14367107, -0.3685745 ,  0.51224554], dtype=float32)>]


#### 학습하기 
* SGD : 확률적 경사 하강법
* 계산된 기울기를 변수인 [W, b]를 갱신하면서 최적의 값을 찾아감

In [83]:
def fit(X, Y, epochs=2000, verbose=100):
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
    
    for i in range(epochs):
        grads = grad_fn(X, Y)  # 기울기 계산 
        optimizer.apply_gradients(zip(grads, variables))  # 갱신 
        
        if(i==0) | ((i+1) % verbose == 0):
            print('Loss at epochs {:5}: {:.5f}'.format(i+1, cost_fn(X, Y).numpy()))        

fit(x_data, y_data)

Loss at epochs     1: 0.15928
Loss at epochs   100: 0.15378
Loss at epochs   200: 0.14859
Loss at epochs   300: 0.14373
Loss at epochs   400: 0.13916
Loss at epochs   500: 0.13487
Loss at epochs   600: 0.13083
Loss at epochs   700: 0.12701
Loss at epochs   800: 0.12340
Loss at epochs   900: 0.11999
Loss at epochs  1000: 0.11676
Loss at epochs  1100: 0.11369
Loss at epochs  1200: 0.11077
Loss at epochs  1300: 0.10800
Loss at epochs  1400: 0.10536
Loss at epochs  1500: 0.10284
Loss at epochs  1600: 0.10044
Loss at epochs  1700: 0.09814
Loss at epochs  1800: 0.09594
Loss at epochs  1900: 0.09384
Loss at epochs  2000: 0.09183


#### Prediction Check

In [84]:
test_data = [[2,1,3,2]] # answer_label [[0,0,1]]
test_data = np.asarray(test_data, dtype=np.float32)

a = hypothesis(test_data)
print(a)
print(tf.argmax(a, 1))

tf.Tensor([[7.4743922e-04 4.7666922e-02 9.5158565e-01]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [85]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))          # 예측값 
print(tf.argmax(y_data, 1))     # 실제값 

tf.Tensor(
[[3.3713121e-08 9.8246746e-05 9.9990165e-01]
 [7.4743817e-04 4.7666889e-02 9.5158565e-01]
 [3.8566617e-10 9.7003169e-02 9.0299690e-01]
 [7.9175763e-08 9.1177320e-01 8.8226721e-02]
 [1.5831372e-01 8.3478457e-01 6.9017108e-03]
 [8.3598949e-02 9.1640097e-01 1.5503129e-07]
 [8.2878691e-01 1.7121208e-01 9.9736792e-07]
 [9.6578580e-01 3.4214135e-02 3.9437280e-09]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
